<a href="https://colab.research.google.com/github/VHEX-LAB/VHEX-Tech/blob/main/client-iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Save/Load models

In [ ]:
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import datasets

iris = datasets.load_iris()
X, y = iris.data, iris.target

model = LogisticRegression(random_state=0, max_iter=1000)
model.fit(X, y)  

In [ ]:
model.predict(X)

In [ ]:
X[0]

In [ ]:
model.predict([X[0]])

In [ ]:
joblib.dump(model, "iris.model")

In [ ]:
model = joblib.load("iris.model")

# Serve models

In [ ]:
!pip install flask-ngrok -q

In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok  # To use flask server in colab

app = Flask(__name__)
run_with_ngrok(app)   
  
@app.route("/")
def hello():
    return "hello"
    
app.run() # Use gunicorn in production evironment

In [ ]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok  # To use flask server in colab

app = Flask(__name__)
run_with_ngrok(app)   
  
@app.route("/", methods=["POST"])
def hello():
    data = request.get_json(True)
    sepal_length = data["sepal_length"]
    sepal_width = data["sepal_width"]
    petal_length = data["petal_length"]
    petal_width = data["petal_width"]

    input = [sepal_length, sepal_width, petal_length, petal_width]
    species = model.predict([input])
    return str(species[0])
    
app.run() # Use gunicorn in production evironment

# Cache(Single machine level)

In [ ]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok  # To use flask server in colab


cache = {}

app = Flask(__name__)
run_with_ngrok(app)   
  
@app.route("/", methods=["POST"])
def hello():
    data = request.get_json(True)
    sepal_length = data["sepal_length"]
    sepal_width = data["sepal_width"]
    petal_length = data["petal_length"]
    petal_width = data["petal_width"]

    key = (sepal_length, sepal_width, petal_length, petal_width)
    if key in cache:
        print("cached")
        return cache[key]

    input = list(key)
    species = model.predict([input])
    cache[key] = str(species[0])
    return str(species[0])

app.run()

# High availability(Single machine Level)

In [ ]:
%%writefile app.py
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from flask import Flask, request
from flask_ngrok import run_with_ngrok  # To use flask server in colab

model = joblib.load("iris.model")
cache = {}

app = Flask(__name__)
  
@app.route("/", methods=["GET", "POST"])
def hello():
    data = request.get_json(True)
    sepal_length = data["sepal_length"]
    sepal_width = data["sepal_width"]
    petal_length = data["petal_length"]
    petal_width = data["petal_width"]

    key = (sepal_length, sepal_width, petal_length, petal_width)
    if key in cache:
        print("cached")
        return cache[key]

    input = list(key)
    species = model.predict([input])
    cache[key] = str(species[0])
    return str(species[0])

In [ ]:
!pip install gunicorn -q

In [ ]:
!gunicorn app:app -w 4

In [ ]:
!nohup gunicorn app:app -w 4 &

In [ ]:
!ps --forest

In [ ]:
!curl 127.0.0.1:8000 \
-H 'Context-Type: application/json' \
--data-raw '{"sepal_length": 1.0,"sepal_width": 2.0,"petal_length": 2.0,"petal_width": 1.0}'

In [ ]:
!kill  336

# BentoML

In [ ]:
!pip install bentoml -q

In [ ]:
%%writefile bento_service.py
import pandas as pd

from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.frameworks.sklearn import SklearnModelArtifact

@env(infer_pip_packages=True)
@artifacts([SklearnModelArtifact('model')])
class IrisClassifier(BentoService):
    @api(input=DataframeInput(), batch=True)
    def predict(self, df: pd.DataFrame):
        return self.artifacts.model.predict(df)

In [ ]:
from bento_service import IrisClassifier

iris_classifier_service = IrisClassifier()
iris_classifier_service.pack('model', model)

In [ ]:
iris_classifier_service.predict([[1.1, 2.3, 2.3, 3.4]])

In [ ]:
iris_classifier_service.start_dev_server()

In [ ]:
!curl 127.0.0.1:5000/predict \
-H 'Context-Type: application/json' \
--data-raw '[{"sepal_length": 1.0,"sepal_width": 2.0,"petal_length": 2.0,"petal_width": 1.0}]'

In [ ]:
iris_classifier_service.stop_dev_server()

In [ ]:
iris_classifier_service.save()

In [ ]:
!bentoml serve IrisClassifier:latest --run-with-ngrok

In [ ]:
!bentoml serve-gunicorn IrisClassifier:latest --workers 4